# Deep Averageing Netrok Analysis

In [26]:
import torch

embd = torch.nn.Embedding(20, 5)
x = torch.tensor([[1, 2, 3], [0, 2, 4]], dtype=torch.int32)
print(x)
out = embd(x)
print(out)
print(out.shape)
print(out.sum(axis=1))
print(out.sum(axis=1).shape)
lenthes = torch.tensor([3, 2])
print(lenthes.shape)
out.sum(axis=1)/lenthes.reshape([-1, 1])

tensor([[1, 2, 3],
        [0, 2, 4]], dtype=torch.int32)
tensor([[[-1.9397,  0.2808,  0.1792, -0.8886, -1.1591],
         [-0.4551, -0.0365,  1.0608, -0.9280,  1.3957],
         [-1.0040, -1.2729, -1.0683, -0.4566,  0.2929]],

        [[ 0.0336,  0.4580, -0.2843, -1.3153,  0.1273],
         [-0.4551, -0.0365,  1.0608, -0.9280,  1.3957],
         [ 0.9413,  1.1517, -1.1855, -0.7650,  1.0793]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 3, 5])
tensor([[-3.3987, -1.0286,  0.1717, -2.2731,  0.5296],
        [ 0.5198,  1.5732, -0.4090, -3.0082,  2.6024]], grad_fn=<SumBackward1>)
torch.Size([2, 5])
torch.Size([2])


tensor([[-1.1329, -0.3429,  0.0572, -0.7577,  0.1765],
        [ 0.2599,  0.7866, -0.2045, -1.5041,  1.3012]], grad_fn=<DivBackward0>)

# Training

## Training with no pretrained Embeddings

In [41]:
!python3  dan.py --num-epochs=20 --checkpoint=10

2
DanModel(
  (embeddings): Embedding(12678, 50, padding_idx=0)
  (linear1): Linear(in_features=50, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=2, bias=True)
  (_softmax): Softmax(dim=1)
  (linear_stack): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=50, out_features=2, bias=True)
  )
)
start epoch 0
number of steps: 10, loss: 0.76076 time: 0.19642
accuracy 0.5874587458745875
number of steps: 20, loss: 0.68439 time: 0.40817
accuracy 0.6171617161716172
start epoch 1
number of steps: 10, loss: 0.74532 time: 0.18718
accuracy 0.6683168316831682
number of steps: 20, loss: 0.65972 time: 0.38661
accuracy 0.698019801980198
start epoch 2
number of steps: 10, loss: 0.70656 time: 0.20366
accuracy 0.7475247524752475
number of steps: 20, loss: 0.61711 time: 0.40451
accuracy 0.7656765676567656
start epoch 3
number of steps: 10, loss: 0.64432 time: 0.17182
accu